In [3]:
from h1st_tune import H1stTunable

In [4]:
import ray

ray.init(ignore_reinit_error=True)

2020-10-26 16:59:54,072	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '192.168.1.2',
 'raylet_ip_address': '192.168.1.2',
 'redis_address': '192.168.1.2:47581',
 'object_store_address': '/tmp/ray/session_2020-10-26_16-59-53_499550_76750/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-26_16-59-53_499550_76750/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2020-10-26_16-59-53_499550_76750',
 'metrics_export_port': 65029}

In [6]:
"""This test checks that BayesOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import time


import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


# def evaluation_fn(step, width, height):
#     return (0.1 + width * step / 100)**(-1) + height * 0.1


# def easy_objective(config):
#     # Hyperparameters
#     width, height = config["width"], config["height"]

#     for step in range(config["steps"]):
#         # Iterative training function - can be any arbitrary training procedure
#         intermediate_score = evaluation_fn(step, width, height)
#         # Feed the score back back to Tune.
#         tune.report(iterations=step, mean_loss=intermediate_score)
#         time.sleep(0.1)

space = {
    "lr": tune.uniform(0.005, 0.05),
    "units": tune.uniform(1, 64),
    "epochs": tune.uniform(4, 50),    
#     "units": tune.randint(1, 64),
#     "n_layer": tune.randint(1, 8)
    "n_layer": tune.quniform(1, 8, 1.0),    
}

algo = BayesOptSearch(   
    utility_kwargs={
    "kind": "ucb",
    "kappa": 2.5,
    "xi": 0.0
})

algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

analysis= tune.run(
    H1stTunable,
    config=space,
    metric="accuracy",
    mode="max",     
#     name="my_exp",
    search_alg=algo,
#     scheduler=scheduler,
    stop={"training_iteration": 5},
    verbose=1,
    num_samples=200
)


Trial name,status,loc,epochs,lr,n_layer,units,acc,iter,total time (s)
H1stTunable_aa2e7258,ERROR,,31.2511,0.00709027,5.25281,11.743,0.815657,4,12.0444
H1stTunable_b9c4ebe8,ERROR,,4.3198,0.0279836,3.92188,14.9928,0.819192,4,6.95301
H1stTunable_b9c53918,ERROR,,9.51381,0.0201927,7.60037,21.3618,0.81697,4,11.1273
H1stTunable_b9c5900c,ERROR,,27.8644,0.0366359,3.54541,62.2223,0.811414,4,11.0112
H1stTunable_b9c5d7f6,ERROR,,48.2726,0.0163302,4.48074,19.9553,0.813131,4,13.86
H1stTunable_b9c61086,ERROR,,17.1027,0.00665991,5.26695,32.6688,0.81404,4,11.1228
H1stTunable_b9c64aba,ERROR,,6.36802,0.0175391,7.35786,16.0924,0.818384,4,10.2572
H1stTunable_c76136f8,RUNNING,192.168.1.2:79450,35.7831,0.0224031,7.55711,9.66382,0.815455,4,15.8583
H1stTunable_c7bdc904,RUNNING,192.168.1.2:79459,19.6891,0.0101063,7.47286,56.2724,0.813939,3,12.9807
H1stTunable_c83d6632,RUNNING,192.168.1.2:79452,15.8653,0.0346993,6.72056,35.9777,0.775556,4,13.6739


(pid=79615) 2020-10-26 20:52:33.198524: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=79615) 2020-10-26 20:52:33.226599: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f92912405a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
(pid=79615) 2020-10-26 20:52:33.226658: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
(pid=79615) INFO:filelock:Lock 10539720656 acquired on ./data/credit_card/data.lock
2020-10-26 20:52:33,479	ERROR trial_runner.py:567 -- Trial H1stTunable_c8a05ea4: Error processing event.
Traceback (most recent call last):
  File "/Users/arimo/Desktop/jupyter/py3_venv/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 535, in _process_trial
    trial.trial_id, result=flat_result)
  File "/Users/arimo/Desktop/jupyter/py3_venv/lib/python3.7/site-packages/r

KeyboardInterrupt: 